In [10]:
### Set up a client for the Search API

In [11]:
# Establishing connection and authorisation:
import os
import tweepy
import pandas as pd
import json

 
# Change the xxx values with your keys
consumer_key = '5CZVKdOnCuVl9zkJ6d3bWv5nL'
consumer_secret = 'nT4rkoT4QSyPIin6Zhzh78Jg6ZXz7YMlVZWIFDKgsDmSqwPwRL'
access_token = '1616344634199838725-8ESR5vU0e5zTOJfpgeMavmEfJ1Yh8j'
access_secret = 'ebHKeGme9Kdak6bLIf1bWSwLVNzNq0aWdoP8ll0iKkNPq'
bearer_token='AAAAAAAAAAAAAAAAAAAAAIG6lQEAAAAAnex9Fahvn0LxVKspzLEMOGaCH6k%3DWUFQkWXsIYHmiDPVzvAqBHUc4nb4ZqWHWyJzyXhxyOR0kXokCI'


client = tweepy.Client(
    wait_on_rate_limit = True,
    consumer_key = consumer_key,
    consumer_secret = consumer_secret,
    access_token = access_token,
    access_token_secret = access_secret,
    bearer_token = bearer_token,
)

None


In [12]:


#Add your credentials here
twitter_keys = {
        'consumer_key':consumer_key,
        'consumer_secret': consumer_secret,
        'access_token_key': access_token,
        'access_token_secret': access_secret
    }

#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [13]:
# Code reproduced from Twitter developer API documentation [4]
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type 

In [22]:
parsedTweets = []

def parse_tweets(status):
    for tweet in status:
        if determine_tweet_type(tweet._json) == 'Retweet':
            if 'extended_tweet' in tweet._json['retweeted_status']:
                full_text = tweet._json['retweeted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['retweeted_status']['text']
               

        elif determine_tweet_type(tweet._json) == 'Quote Tweet':
            if 'extended_tweet' in tweet._json['quoted_status']:
                full_text = tweet._json['quoted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['quoted_status']['text']
                
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
            else:
                full_text = tweet._json['text']

        mydict = { "tweet_id": tweet._json["id_str"],
                      "user_id":tweet._json_json['user']['id']
                       "date":tweet._json["created_at"],
                       "full_text": full_text,
                       "tweet_type": determine_tweet_type(tweet._json),
                        "reply_count": tweet._json["reply_count"], #Number of times Tweet has been replied to
                        "place":tweet._json["place"], # place linked to the tweet 
                        "coordinartes":tweet._json["coordinates"], #location reported by the app
                        "likes":tweet._json["favorite_count"], #number of likes 
                       "quote_count": tweet._json["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet._json["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet._json["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"]
              }
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list           

In [23]:
parsedTweets = []

for page in tweepy.Cursor(api.search_30_day,label='enviroment', 
                          query="avatar",
                          fromDate='202302010000',
                          toDate=  '202303010000').pages(1):
    
    parse_tweets(page)
    
    
tweetdf = pd.DataFrame(parsedTweets)

In [26]:
objeto_tw =api.search_30_day(label='enviroment',
query="avatar")

    
    

In [38]:
# 'user': {'id': 1598268594298798081, 'id_str': '1598268594298798081', 'name': 'CanaBoyz'
#for o in objeto_tw:
#    print(o._json['user']['name'])
objeto_tw    

[Status(_api=<tweepy.api.API object at 0x000001CEF5AC7D30>, _json={'created_at': 'Fri Mar 03 12:43:41 +0000 2023', 'id': 1631636463581294592, 'id_str': '1631636463581294592', 'text': "RT @SeedifyFund: Couldn't be more excited for the upcoming Mounts #NFT sale&amp;mint tomorrow! Here's the timeline:\n\n🕰️ Phase 1: 9 am - 3 pm UT…", 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1598268594298798081, 'id_str': '1598268594298798081', 'name': 'CanaBoyz', 'screen_name': 'club1112022', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'verified_type': 'blue', 'followers_count': 219, 'friends_count': 620, 'listed_count': 1, 'favourites_count': 1578, 'statuses_count': 2186, 'created_at': 'Thu Dec 0

In [21]:
user = api.get_user(screen_name='twitter')


print(api.rate_limit_status())

{'rate_limit_context': {'access_token': '1616344634199838725-8ESR5vU0e5zTOJfpgeMavmEfJ1Yh8j'}, 'resources': {'lists': {'/lists/list': {'limit': 15, 'remaining': 15, 'reset': 1677846112}, '/lists/:id/tweets&GET': {'limit': 900, 'remaining': 900, 'reset': 1677846112}, '/lists/:id/followers&GET': {'limit': 180, 'remaining': 180, 'reset': 1677846112}, '/lists/memberships': {'limit': 75, 'remaining': 75, 'reset': 1677846112}, '/lists/:id&DELETE': {'limit': 300, 'remaining': 300, 'reset': 1677846112}, '/lists/subscriptions': {'limit': 15, 'remaining': 15, 'reset': 1677846112}, '/lists/members': {'limit': 900, 'remaining': 900, 'reset': 1677846112}, '/lists/:id&GET': {'limit': 75, 'remaining': 75, 'reset': 1677846112}, '/lists/subscribers/show': {'limit': 15, 'remaining': 15, 'reset': 1677846112}, '/lists/:id&PUT': {'limit': 300, 'remaining': 300, 'reset': 1677846112}, '/lists/show': {'limit': 75, 'remaining': 75, 'reset': 1677846112}, '/lists/ownerships': {'limit': 15, 'remaining': 15, 'rese